In [381]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [382]:
# load data https://www.kaggle.com/datasets/datafiniti/consumer-reviews-of-amazon-products?select=1429_1.csv
csv_path = "data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"
df = pd.read_csv(csv_path)
df

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,http://reviews.bestbuy.com/3545/5620410/review...
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review

In [383]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

In [384]:
#looking at different attributes
print('brand')
print(df['brand'].value_counts())
print()
print('reviews.didPurchase')
print(df['reviews.didPurchase'].value_counts())
print()
print('reviews.doRecommend')
print(df['reviews.doRecommend'].value_counts())
print()
print('reviews.id')
print(df['reviews.id'].unique().size)

brand
Amazon          16153
Amazonbasics    12169
AmazonBasics       10
Name: brand, dtype: int64

reviews.didPurchase
True     8
False    1
Name: reviews.didPurchase, dtype: int64

reviews.doRecommend
True     15353
False      733
Name: reviews.doRecommend, dtype: int64

reviews.id
42


In [385]:
# Attribute Removal
# dropping unuseful cols, because they mainly contain null values
# dateSeen only specifies the date when the review was scraped
unuseful_cols = ['reviews.didPurchase', 'reviews.id', 'keys', 'reviews.dateSeen', 'imageURLs', 'sourceURLs', 'reviews.sourceURLs', 'dateUpdated']
df = df.drop(unuseful_cols, axis=1)
# rename columns to make them uniform and more self-explanatory (see DataFinity for documentation)
df = df.rename(columns={
    'dateAdded': 'productDateAdded',
    'name': 'productName',
    'reviews.date': 'reviewDate',
    'reviews.doRecommend': 'reviewDoRecommend',
    'reviews.numHelpful': 'reviewNumHelpful',
    'reviews.rating': 'reviewRating',
    'reviews.text': 'reviewText',
    'reviews.title': 'reviewTitle',
    'reviews.username': 'reviewUsername'
})
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,reviewDoRecommend,reviewNumHelpful,reviewRating,reviewText,reviewTitle,reviewUsername
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02T00:00:00.000Z,NaN,NaN,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31T00:00:00.000Z,NaN,NaN,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06T00:00:00.000Z,NaN,NaN,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28T00:00:00.000Z,NaN,NaN,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22T00:00:00.000Z,NaN,NaN,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07T00:00:00.000Z,True,0.0,5,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20T00:00:00.000Z,True,0.0,4,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30T00:00:00.000Z,True,0.0,5,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17T00:00:00.000Z,True,0.0,5,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ


In [386]:
# Data Cleansing
# set NaN values to 0
df['reviewNumHelpful'] = df['reviewNumHelpful'].fillna(0).astype('int16')
df['reviewDoRecommend'] = df['reviewDoRecommend'].fillna(0).astype('int16')
# set data types
df['reviewRating'] = df['reviewRating'].astype('int8')
df['reviewDate'] = pd.to_datetime(df['reviewDate']).dt.date
df['productDateAdded'] = pd.to_datetime(df['productDateAdded']).dt.date
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,reviewDoRecommend,reviewNumHelpful,reviewRating,reviewText,reviewTitle,reviewUsername
0,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02,0,0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang
1,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31,0,0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG
2,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06,0,0,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert
3,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28,0,0,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson
4,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22,0,0,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07,1,0,5,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20,1,0,4,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30,1,0,5,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17,1,0,5,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ


In [387]:
# check for duplicates
df.duplicated().sum()

0

In [388]:
df['reviewRating'].describe()

count    28332.000000
mean         4.514048
std          0.934957
min          1.000000
25%          4.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: reviewRating, dtype: float64

In [389]:
df['reviewDoRecommend'].describe()

count    28332.000000
mean         0.541896
std          0.498250
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: reviewDoRecommend, dtype: float64

In [390]:
df['reviewDate'].describe()

count          28332
unique          1275
top       2017-01-23
freq             532
Name: reviewDate, dtype: object

In [391]:
# Text Pre_processing / Additional Attributes
df['reviewTextLength'] = df['reviewText'].str.len()
df['reviewTitleLength'] = df['reviewTitle'].str.len()
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,reviewDoRecommend,reviewNumHelpful,reviewRating,reviewText,reviewTitle,reviewUsername,reviewTextLength,reviewTitleLength
0,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02,0,0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,139,88
1,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31,0,0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,67,57
2,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06,0,0,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,56,50
3,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28,0,0,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,67,52
4,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22,0,0,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,57,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07,1,0,5,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,116,9
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20,1,0,4,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21,83,25
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30,1,0,5,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious,317,28
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17,1,0,5,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,241,44


In [392]:
# Data Scaling
review_text_length = df['reviewTextLength']
review_title_length = df['reviewTitleLength']
review_num_helpful = df['reviewNumHelpful']
min_max_scaler = preprocessing.MinMaxScaler()
# transform the column with log, to mend outliers with very long review texts
review_text_length = np.log(review_text_length + 1)
review_num_helpful = np.log(review_num_helpful + 1)
review_text_length_scaled = min_max_scaler.fit_transform(review_text_length.values.reshape(-1, 1))
review_title_length_scaled = min_max_scaler.fit_transform(review_title_length.values.reshape(-1, 1))
review_num_helpful = min_max_scaler.fit_transform(review_num_helpful.values.reshape(-1, 1))
df['reviewTextLength'] = review_text_length_scaled
df['reviewTitleLength'] = review_title_length_scaled
df['reviewNumHelpful'] = review_num_helpful
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,reviewDoRecommend,reviewNumHelpful,reviewRating,reviewText,reviewTitle,reviewUsername,reviewTextLength,reviewTitleLength
0,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02,0,0.0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,0.509589,0.621429
1,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31,0,0.0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,0.422972,0.400000
2,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06,0,0.0,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,0.401806,0.350000
3,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28,0,0.0,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,0.422972,0.364286
4,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22,0,0.0,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,0.403892,0.385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07,1,0.0,5,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,0.488062,0.057143
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20,1,0.0,4,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21,0.448317,0.171429
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30,1,0.0,5,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious,0.607993,0.192857
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17,1,0.0,5,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,0.575234,0.307143


In [393]:
# Data Transformation
# add review count to each author and scale it
review_counts = df['reviewUsername'].value_counts()
df['reviewUsernameReviewCount'] = df['reviewUsername'].map(review_counts)
review_counts = df['reviewUsernameReviewCount']
review_counts = np.log(review_counts + 1)
review_counts_scaled = min_max_scaler.fit_transform(review_counts.values.reshape(-1, 1))
df['reviewUsernameReviewCount'] = review_counts_scaled
# calculate days between productAdded and reviewDate
df['daysBetweenProductAddedAndReview'] = (df['reviewDate'] - df['productDateAdded']).dt.days
# encode category columns to binary columns
encoded_cols = df['primaryCategories'].str.get_dummies(sep=',')
df = pd.concat([df, encoded_cols], axis=1)
encoded_cols = df['categories'].str.get_dummies(sep=',')
df = pd.concat([df, encoded_cols], axis=1)
df.columns = df.columns.str.replace(' ', '_')
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,...,Wired_Headsets_&_Speakers,Wireless_Speakers,all_electronics,computers,eBook_Readers,eBook_Readers_&_Accessories,iPad_&_Tablets,iPad_Accessories,iPads_Tablets,iPod
0,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02,...,0,0,0,0,0,0,0,0,0,0
1,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31,...,0,0,0,0,0,0,0,0,0,0
2,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06,...,0,0,0,0,0,0,0,0,0,0
3,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28,...,0,0,0,0,0,0,0,0,0,0
4,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07,...,0,0,0,0,0,0,0,0,0,0
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20,...,0,0,0,0,0,0,0,0,0,0
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30,...,0,0,0,0,0,0,0,0,0,0
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17,...,0,0,0,0,0,0,0,0,0,0


In [394]:
# incorrect dateAdded
df['daysBetweenProductAddedAndReview'].describe()

count    28332.000000
mean       134.126677
std        224.069334
min      -2895.000000
25%        -35.000000
50%         77.500000
75%        288.000000
max       1614.000000
Name: daysBetweenProductAddedAndReview, dtype: float64

# Data Exploration Moritz

In [395]:
df

,id,productDateAdded,productName,asins,brand,categories,primaryCategories,manufacturer,manufacturerNumber,reviewDate,...,Wired_Headsets_&_Speakers,Wireless_Speakers,all_electronics,computers,eBook_Readers,eBook_Readers_&_Accessories,iPad_&_Tablets,iPad_Accessories,iPads_Tablets,iPod
0,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-03-02,...,0,0,0,0,0,0,0,0,0,0
1,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-08-31,...,0,0,0,0,0,0,0,0,0,0
2,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-12-06,...,0,0,0,0,0,0,0,0,0,0
3,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2016-03-28,...,0,0,0,0,0,0,0,0,0,0
4,AVpgNzjwLJeJML43Kpxn,2015-10-30,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,AmazonBasics,HL-002619,2017-04-22,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2016-12-07,...,0,0,0,0,0,0,0,0,0,0
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-20,...,0,0,0,0,0,0,0,0,0,0
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-01-30,...,0,0,0,0,0,0,0,0,0,0
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,Amazon,53-004485,2017-02-17,...,0,0,0,0,0,0,0,0,0,0


In [396]:
df['reviewRating'].value_counts().plot.bar()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
df["productName"].unique()

In [ ]:
df["reviewUsername"].value_counts()